# purpose : New pretrained file for DeepgazeIIE model 

## data & frame 준비 

In [1]:
# data path
base_path = '/Users/krc/Documents/retail/'

In [2]:
# install check 
import torch
import os
import csv
torch.__version__

'1.13.0.dev20220623'

In [3]:
os.getcwd()

'/Users/krc/Documents/retail/retail_gh/deepgaze_sample'

In [4]:
%cd ../../

/Users/krc/Documents/retail


In [10]:
# #mmdetection installation 
# !pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu113/torch1.12.0/index.html
# !git clone https://github.com/open-mmlab/mmdetection.git
# %cd mmdetection
# !pip install -r requirements/build.txt
# !pip install -v -e .
# !pip install mmdet

Looking in links: https://download.openmmlab.com/mmcv/dist/cu113/torch1.12.0/index.html
Cloning into 'mmdetection'...
remote: Enumerating objects: 31884, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 31884 (delta 0), reused 4 (delta 0), pack-reused 31871
Receiving objects: 100% (31884/31884), 40.80 MiB | 16.91 MiB/s, done.
Resolving deltas: 100% (22972/22972), done.
/Users/krc/Documents/retail/mmdetection
Using pip 22.1.2 from /Users/krc/miniforge3/envs/pytorch/lib/python3.8/site-packages/pip (python 3.8)
Obtaining file:///Users/krc/Documents/retail/mmdetection
  Running command python setup.py egg_info
  running egg_info
  creating /private/var/folders/5m/j1nv4tbn34l6v5n5hjqzx0c40000gn/T/pip-pip-egg-info-g383yw8b/mmdet.egg-info
  writing /private/var/folders/5m/j1nv4tbn34l6v5n5hjqzx0c40000gn/T/pip-pip-egg-info-g383yw8b/mmdet.egg-info/PKG-INFO
  writing dependency_links to /private/var/folders/5m/j1nv4tbn34l6v5n5hjqzx

In [11]:
os.listdir()

['demo',
 'tools',
 'docker',
 'pytest.ini',
 'LICENSE',
 'requirements.txt',
 '.pre-commit-config.yaml',
 'CITATION.cff',
 '.owners.yml',
 'resources',
 'tests',
 'MANIFEST.in',
 'requirements',
 'docs',
 'README_zh-CN.md',
 '.readthedocs.yml',
 'README.md',
 'setup.py',
 '.gitignore',
 'mmdet.egg-info',
 'configs',
 '.github',
 '.dev_scripts',
 'setup.cfg',
 '.git',
 'model-index.yml',
 '.circleci',
 'mmdet']

In [5]:
# annotation by csv 
f =open('/Users/krc/Documents/retail/data/SKU110K_fixed/annotations/annotations_train.csv','r')
annotation = csv.reader(f)
# image_name,x1,y1,x2,y2,class,image_width,image_height
ls_ann = list(annotation)
print(ls_ann)
f.close()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [6]:
index = ['image_name','x1','y1','x2','y2','class','image_width','image_height']
print(index)
print(ls_ann[-1])
print(ls_ann[-1][0])


['image_name', 'x1', 'y1', 'x2', 'y2', 'class', 'image_width', 'image_height']
['train_999.jpg', '905', '1589', '1144', '1712', 'object', '2336', '4160']
train_999.jpg


In [7]:
!pip install PyYAML

In [43]:
# annotation 생성
import copy
import os.path as osp
import cv2
import yaml

import mmcv
import numpy as np

from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset

CLASSES = ('object')
# cat2label = {k:i for i, k in enumerate(CLASSES)}

# 반드시 아래 Decorator 설정 할것.@DATASETS.register_module() 설정 시 force=True를 입력하지 않으면 Dataset 재등록 불가. 
# @DATASETS.register_module(force=True)
# class SKIKU110K(CustomDataset):
#   CLASSES = ('object')
  
  ##### self.data_root: ./kitti_tiny/ self.ann_file: ./kitti_tiny/train.txt self.img_prefix: ./kitti_tiny/training/image_2
  #### ann_file: ./kitti_tiny/train.txt
  # annotation에 대한 모든 파일명을 가지고 있는 텍스트 파일을 __init__(self, ann_file)로 입력 받고, 이 self.ann_file이 load_annotations()의 인자로 입력
def load_annotations(self, ann_file):
#     print('##### self.data_root:', self.data_root, 'self.ann_file:', self.ann_file, 'self.img_prefix:', self.img_prefix)
#     print('#### ann_file:', ann_file)
#    cat2label = {k:i for i, k in enumerate(self.CLASSES)}
    # 포맷 중립 데이터를 담을 list 객체
    data_infos = []
    
    for label in ann_file:
      filename = label[0]
      height, width = label[6],label[7]
      # 개별 image의 annotation 정보 저장용 Dict 생성. key값 filename 에는 image의 파일명만 들어감(디렉토리는 제외)
      data_info = {'filename': str(filename),
                   'width': width, 'height': height}

      # 오브젝트의 클래스명은 bbox_names로 저장. 
      bbox_names = label[5]
      # bbox 좌표를 저장
      bboxes = [int(label[1]),int(label[2]), int(label[3]) -int(label[1]), int(label[4])-int(label[2])]

# 굳이? 
#       # 클래스명이 해당 사항이 없는 대상 Filtering out, 'DontCare'sms ignore로 별도 저장.
#       gt_bboxes = []
#       gt_labels = []
#       gt_bboxes_ignore = []
#       gt_labels_ignore = []

#       for bbox_name, bbox in zip(bbox_names, bboxes):
#         # 만약 bbox_name이 클래스명에 해당 되면, gt_bboxes와 gt_labels에 추가, 그렇지 않으면 gt_bboxes_ignore, gt_labels_ignore에 추가
#         if bbox_name in cat2label:
#           gt_bboxes.append(bbox)
#           # gt_labels에는 class id를 입력
#           gt_labels.append(cat2label[bbox_name])
#         else:
#           gt_bboxes_ignore.append(bbox)
#           gt_labels_ignore.append(-1)
      # 개별 image별 annotation 정보를 가지는 Dict 생성. 해당 Dict의 value값은 모두 np.array임. 
      data_anno = {
          'bboxes': np.array(bboxes, dtype=np.float32).reshape(-1, 4).tolist(),
          'labels':bbox_names,
      }
      # image에 대한 메타 정보를 가지는 data_info Dict에 'ann' key값으로 data_anno를 value로 저장. 
      data_info.update(ann=data_anno)
      # 전체 annotation 파일들에 대한 정보를 가지는 data_infos에 data_info Dict를 추가
      data_infos.append(data_info)

    return data_infos

In [44]:

new_ann = load_annotations(CustomDataset,ann_file=ls_ann)
print(new_ann)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [52]:
new_ann[0]['ann']

{'bboxes': [[208.0, 537.0, 214.0, 277.0]], 'labels': 'object'}

In [53]:
import json

with open('new_ann.json', 'w') as f:
    json.dump(new_ann, f)

In [54]:
os.getcwd()

'/Users/krc/Documents/retail'